In [1]:
import numpy

In [2]:
import tensorflow as tf

In [3]:
import keras

In [4]:
tf.__version__

'2.3.1'

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [13]:
tf.config.list_physical_devices('XLA_GPU')

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [11]:
tf.test.is_gpu_available()

True

### demo of using 1. GPU and 2. balancing generator

In [10]:
def generator(features, labels, batch_size, chans, samples, keep_order=False):

    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros([batch_size, 1, chans, samples])
    batch_labels = np.zeros([batch_size])

    while True:
        for idx_label in np.arange(1000, features.shape[0]-batch_size, batch_size):
            for i in range(batch_size):

                for ch in range(chans):
                    batch_features[i,0,ch,:] = features[(idx_label+i-samples):i+idx_label,ch]
                batch_labels[i] = labels[idx_label+i]

            if keep_order == False:
                dat_ = (batch_features, batch_labels)
                if np.where(dat_[1] > 0.05)[0].shape[0] == 0:
                    continue
                num_pos = int(dat_[1].shape[0] / np.where(dat_[1] > 0.05)[0].shape[0])
                ind_label = np.concatenate([np.where(dat_[1]>0.05)[0] for i in range(num_pos)])
                ind_no_label = np.where(dat_[1]<0.05)[0][:dat_[1].shape[0]-ind_label.shape[0]] # 0 line is not exactly zero
                np.random.shuffle(ind_label)
                np.random.shuffle(ind_no_label)

                batch_labels_ = np.concatenate((dat_[1][ind_no_label], dat_[1][ind_label]), axis=0)
                batch_features_ = np.concatenate((dat_[0][ind_no_label,:,:,:], dat_[0][ind_label,:,:,:]), axis=0)

                yield batch_features_, batch_labels_, [None]
            else:
                yield batch_features, batch_labels, [None]


In [ ]:
gen_tr = generator(X_train, y_train, batch_size, chans, samples)
gen_val = generator(X_val, y_val, batch_size, chans, samples)